<a href="https://colab.research.google.com/github/MarkAvilin1/DS-and-ML/blob/main/text_segmrntation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Task Ultra Pro**
Сегментация текста

In [ ]:
!pip install -q pymorphy2

     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 8.2 MB 9.9 MB/s 


In [ ]:
from tensorflow.keras.layers import Dense, Embedding, Input, concatenate, Activation, MaxPooling1D, Conv1D, BatchNormalization, Dropout, Conv2DTranspose, Conv1DTranspose, Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam, Adadelta
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import backend as K
from nltk.stem import WordNetLemmatizer
from tensorflow.keras import utils
from gensim.models import word2vec
import matplotlib.pyplot as plt
from google.colab import files
import tensorflow as tf
import pandas as pd
import numpy as np
import pymorphy2 
import nltk
import re
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
data = pd.read_csv('Train.csv', sep=',')

In [ ]:
print(data.columns)

Index(['ID', 'Gender', 'Ever_Married', 'Age', 'Graduated', 'Profession',
       'Work_Experience', 'Spending_Score', 'Family_Size', 'Var_1',
       'Segmentation'],
      dtype='object')


In [ ]:
print(data.dtypes)

ID                   int64
Gender              object
Ever_Married        object
Age                  int64
Graduated           object
Profession          object
Work_Experience    float64
Spending_Score      object
Family_Size        float64
Var_1               object
Segmentation        object
dtype: object


In [ ]:
drop_list = ['ID', 'Age', 'Work_Experience', 'Family_Size']
data = data.drop(drop_list, axis=1)

In [ ]:
print(data.dtypes)
print(data.shape)
print(data[:10])

Gender            object
Ever_Married      object
Graduated         object
Profession        object
Spending_Score    object
Var_1             object
Segmentation      object
dtype: object
(8068, 7)
   Gender Ever_Married Graduated  ... Spending_Score  Var_1 Segmentation
0    Male           No        No  ...            Low  Cat_4            D
1  Female          Yes       Yes  ...        Average  Cat_4            A
2  Female          Yes       Yes  ...            Low  Cat_6            B
3    Male          Yes       Yes  ...           High  Cat_6            B
4  Female          Yes       Yes  ...           High  Cat_6            A
5    Male          Yes        No  ...        Average  Cat_6            C
6    Male           No       Yes  ...            Low  Cat_6            C
7  Female           No       Yes  ...            Low  Cat_6            D
8  Female          Yes       Yes  ...            Low  Cat_7            D
9  Female          Yes       Yes  ...        Average  Cat_6            

In [ ]:
segments = ['Healthcare', 'Engineer', 'Lawyer', 'Doctor', 'Artist', 'Homemaker']
new_pro = []
for i in data['Profession']:
  if i in segments:
    new_pro.append(f"<s{segments.index(i) + 1}> {i} </s{segments.index(i) + 1}>")
  else:
    new_pro.append(i)
print(len(new_pro))
print(new_pro[:20])

8068
['<s1> Healthcare </s1>', '<s2> Engineer </s2>', '<s2> Engineer </s2>', '<s3> Lawyer </s3>', 'Entertainment', '<s5> Artist </s5>', '<s1> Healthcare </s1>', '<s1> Healthcare </s1>', '<s2> Engineer </s2>', '<s5> Artist </s5>', '<s2> Engineer </s2>', '<s1> Healthcare </s1>', 'Executive', '<s3> Lawyer </s3>', '<s4> Doctor </s4>', '<s1> Healthcare </s1>', '<s6> Homemaker </s6>', '<s1> Healthcare </s1>', 'Entertainment', '<s5> Artist </s5>']


In [ ]:
data['Profession'].update(new_pro)

In [ ]:
for i in data['Profession'][:30]:
  print(i)

<s1> Healthcare </s1>
<s2> Engineer </s2>
<s2> Engineer </s2>
<s3> Lawyer </s3>
Entertainment
<s5> Artist </s5>
<s1> Healthcare </s1>
<s1> Healthcare </s1>
<s2> Engineer </s2>
<s5> Artist </s5>
<s2> Engineer </s2>
<s1> Healthcare </s1>
Executive
<s3> Lawyer </s3>
<s4> Doctor </s4>
<s1> Healthcare </s1>
<s6> Homemaker </s6>
<s1> Healthcare </s1>
Entertainment
<s5> Artist </s5>
<s6> Homemaker </s6>
<s1> Healthcare </s1>
<s5> Artist </s5>
<s5> Artist </s5>
Executive
<s5> Artist </s5>
<s1> Healthcare </s1>
<s5> Artist </s5>
<s5> Artist </s5>
<s5> Artist </s5>


In [ ]:
data_list = np.array(data)

In [ ]:
print(data_list[:20])
print(data_list.shape)

[['Male' 'No' 'No' '<s1> Healthcare </s1>' 'Low' 'Cat_4' 'D']
 ['Female' 'Yes' 'Yes' '<s2> Engineer </s2>' 'Average' 'Cat_4' 'A']
 ['Female' 'Yes' 'Yes' '<s2> Engineer </s2>' 'Low' 'Cat_6' 'B']
 ['Male' 'Yes' 'Yes' '<s3> Lawyer </s3>' 'High' 'Cat_6' 'B']
 ['Female' 'Yes' 'Yes' 'Entertainment' 'High' 'Cat_6' 'A']
 ['Male' 'Yes' 'No' '<s5> Artist </s5>' 'Average' 'Cat_6' 'C']
 ['Male' 'No' 'Yes' '<s1> Healthcare </s1>' 'Low' 'Cat_6' 'C']
 ['Female' 'No' 'Yes' '<s1> Healthcare </s1>' 'Low' 'Cat_6' 'D']
 ['Female' 'Yes' 'Yes' '<s2> Engineer </s2>' 'Low' 'Cat_7' 'D']
 ['Female' 'Yes' 'Yes' '<s5> Artist </s5>' 'Average' 'Cat_6' 'C']
 ['Female' 'No' 'Yes' '<s2> Engineer </s2>' 'Low' 'Cat_6' 'A']
 ['Male' 'No' 'No' '<s1> Healthcare </s1>' 'Low' 'Cat_4' 'D']
 ['Female' 'No' 'No' 'Executive' 'Low' 'Cat_3' 'D']
 ['Male' 'Yes' 'No' '<s3> Lawyer </s3>' 'Low' 'Cat_6' 'A']
 ['Female' 'Yes' 'No' '<s4> Doctor </s4>' 'Low' 'Cat_3' 'B']
 ['Female' 'No' 'No' '<s1> Healthcare </s1>' 'Low' 'Cat_1' 'C']
 ['F

In [ ]:
new_file = []
for lst in data_list:
  for i in str(lst).split():
    new_file.append(i)

In [ ]:
for i in range(len(new_file)):
  new_file[i] = new_file[i].replace("'", '')
  new_file[i] = new_file[i].replace("[", '')
  new_file[i] = new_file[i].replace("]", '')

In [ ]:
print(new_file[:9])
print(len(new_file))

['Male', 'No', 'No', '<s1>', 'Healthcare', '</s1>', 'Low', 'Cat_4', 'D']
68684


In [ ]:
docTrain = new_file[:60001]
docTest = new_file[60001:]

In [ ]:
tokenizer = Tokenizer(lower=True, filters = '', char_level=False)

tokenizer.fit_on_texts(new_file) 
clean_voc = {}                    

for item in tokenizer.word_index.items(): 
  clean_voc[item[0]] = item[1]

In [ ]:
print('Словарь все слов по их частотноти:') 
print(clean_voc)
print('Длина словаря:', len(clean_voc))

tag = 'artist'
print('Индекс тега', tag, ':' ,clean_voc[tag])

Словарь все слов по их частотноти:
{'yes': 1, 'no': 2, 'cat_6': 3, 'low': 4, 'male': 5, 'female': 6, '<s5>': 7, 'artist': 8, '</s5>': 9, 'd': 10, 'average': 11, 'a': 12, 'c': 13, 'b': 14, '<s1>': 15, 'healthcare': 16, '</s1>': 17, 'high': 18, 'cat_4': 19, 'entertainment': 20, 'cat_3': 21, '<s2>': 22, 'engineer': 23, '</s2>': 24, '<s4>': 25, 'doctor': 26, '</s4>': 27, '<s3>': 28, 'lawyer': 29, '</s3>': 30, 'executive': 31, 'cat_2': 32, 'nan': 33, 'marketing': 34, '<s6>': 35, 'homemaker': 36, '</s6>': 37, 'cat_7': 38, 'cat_1': 39, 'cat_5': 40}
Длина словаря: 40
Индекс тега artist : 8


In [ ]:
tok_agreem = tokenizer.texts_to_sequences(docTrain) 
print("Взгляните на фрагмент обучающей выборки:")
print("Исходный текст:                                 ", docTrain[:200])
print("Тот же текст, но как последовательность индексов: ", tok_agreem[:200])
print(len(tok_agreem))

Взгляните на фрагмент обучающей выборки:
Исходный текст:                                  ['Male', 'No', 'No', '<s1>', 'Healthcare', '</s1>', 'Low', 'Cat_4', 'D', 'Female', 'Yes', 'Yes', '<s2>', 'Engineer', '</s2>', 'Average', 'Cat_4', 'A', 'Female', 'Yes', 'Yes', '<s2>', 'Engineer', '</s2>', 'Low', 'Cat_6', 'B', 'Male', 'Yes', 'Yes', '<s3>', 'Lawyer', '</s3>', 'High', 'Cat_6', 'B', 'Female', 'Yes', 'Yes', 'Entertainment', 'High', 'Cat_6', 'A', 'Male', 'Yes', 'No', '<s5>', 'Artist', '</s5>', 'Average', 'Cat_6', 'C', 'Male', 'No', 'Yes', '<s1>', 'Healthcare', '</s1>', 'Low', 'Cat_6', 'C', 'Female', 'No', 'Yes', '<s1>', 'Healthcare', '</s1>', 'Low', 'Cat_6', 'D', 'Female', 'Yes', 'Yes', '<s2>', 'Engineer', '</s2>', 'Low', 'Cat_7', 'D', 'Female', 'Yes', 'Yes', '<s5>', 'Artist', '</s5>', 'Average', 'Cat_6', 'C', 'Female', 'No', 'Yes', '<s2>', 'Engineer', '</s2>', 'Low', 'Cat_6', 'A', 'Male', 'No', 'No', '<s1>', 'Healthcare', '</s1>', 'Low', 'Cat_4', 'D', 'Female', 'No', 'No', 'Executive', 

In [ ]:
tags_index = ['<s' + str(i) + '>' for i in range(1, 7)] 
closetags = ['</s' + str(i) + '>' for i in range(1, 7)] 
tags_index.extend(closetags) 

tags_index = np.array([clean_voc[i] for i in tags_index]) 
print('Индексы всех тегов:', tags_index)

Индексы всех тегов: [15 22 28 25  7 35 17 24 30 27  9 37]


In [ ]:
print(closetags)

['</s1>', '</s2>', '</s3>', '</s4>', '</s5>', '</s6>']


In [ ]:
seg_idx = np.array([clean_voc[i.lower()] for i in segments])
print(seg_idx)

[16 23 29 26  8 36]


In [ ]:
def getSamples(tok_agreem, tags_index, seg_idx):
  xTrain = []
  yTrain = []
  vectors = [[1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 0, 1]]
  for i in range(len(tok_agreem)):
    if tok_agreem[i] in tags_index:
      pass
    elif tok_agreem[i] in seg_idx:
      yTrain.append(vectors[list(seg_idx).index(tok_agreem[i])])
      xTrain.append(tok_agreem[i])
    else:
      xTrain.append(tok_agreem[i])
      yTrain.append([0, 0, 0, 0, 0, 0])
  return xTrain, yTrain

In [ ]:
xData, yData = getSamples(tok_agreem, tags_index, seg_idx)

In [ ]:
print(xData, '\n', len(xData))
print(yData, '\n', len(yData))

[[5], [2], [2], [16], [4], [19], [10], [6], [1], [1], [23], [11], [19], [12], [6], [1], [1], [23], [4], [3], [14], [5], [1], [1], [29], [18], [3], [14], [6], [1], [1], [20], [18], [3], [12], [5], [1], [2], [8], [11], [3], [13], [5], [2], [1], [16], [4], [3], [13], [6], [2], [1], [16], [4], [3], [10], [6], [1], [1], [23], [4], [38], [10], [6], [1], [1], [8], [11], [3], [13], [6], [2], [1], [23], [4], [3], [12], [5], [2], [2], [16], [4], [19], [10], [6], [2], [2], [31], [4], [21], [10], [5], [1], [2], [29], [4], [3], [12], [6], [1], [2], [26], [4], [21], [14], [6], [2], [2], [16], [4], [39], [13], [6], [2], [2], [36], [4], [21], [10], [5], [2], [2], [16], [4], [3], [14], [5], [1], [1], [20], [11], [3], [14], [6], [1], [1], [8], [18], [3], [13], [5], [1], [1], [36], [4], [21], [12], [6], [2], [2], [16], [4], [3], [10], [5], [1], [1], [8], [4], [21], [12], [6], [2], [1], [8], [4], [3], [14], [6], [33], [2], [31], [11], [21], [14], [5], [1], [2], [8], [11], [3], [13], [5], [2], [2], [16], [

In [ ]:
def tegDeleter(docTrain):
  docs = []
  tegs = ['<s1>', '<s2>', '<s3>', '<s4>', '<s5>', '<s6>', '</s1>', '</s2>', '</s3>', '</s4>', '</s5>', '</s6>']
  for i in docTrain:
    if i in tegs:
      pass
    else:
      docs.append(i)
  return docs 

In [ ]:
decoded_text = tegDeleter(docTrain)

In [ ]:
print('длина xData:', len(xData))
print('длина yData:', len(yData))
print('Сдекодированые слова:', decoded_text[50:90])
print('Часть xData:     ', xData[50:90])
print('Часть yData:     ', yData[50:90])

длина xData: 49321
длина yData: 49321
Сдекодированые слова: ['No', 'Yes', 'Healthcare', 'Low', 'Cat_6', 'D', 'Female', 'Yes', 'Yes', 'Engineer', 'Low', 'Cat_7', 'D', 'Female', 'Yes', 'Yes', 'Artist', 'Average', 'Cat_6', 'C', 'Female', 'No', 'Yes', 'Engineer', 'Low', 'Cat_6', 'A', 'Male', 'No', 'No', 'Healthcare', 'Low', 'Cat_4', 'D', 'Female', 'No', 'No', 'Executive', 'Low', 'Cat_3']
Часть xData:      [[2], [1], [16], [4], [3], [10], [6], [1], [1], [23], [4], [38], [10], [6], [1], [1], [8], [11], [3], [13], [6], [2], [1], [23], [4], [3], [12], [5], [2], [2], [16], [4], [19], [10], [6], [2], [2], [31], [4], [21]]
Часть yData:      [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0], [

In [ ]:
def getSetFromIndices(wordIndices, xLen, step): 
  xBatch = [] 
  wordsLen = len(wordIndices) 
  index = 0 
  while (index + xLen <= wordsLen): 
    xBatch.append(wordIndices[index:index+xLen]) 
    index += step 
  return xBatch

In [ ]:
xLen = 256
step = 30
embeddingSize = 300

In [ ]:
xTrain = getSetFromIndices(decoded_text, xLen, step) 
yTrain = getSetFromIndices(yData, xLen, step)

In [ ]:
print('Длина xTrain:', len(xTrain))
print('Длина yTrain:', len(yTrain))
print('Длина примера из xTrain:',len(xTrain[0]))
print('Длина примера из yTrain:',len(yTrain[0]), '\n')
print('Пример xTrain', xTrain[0])
print('Пример yTrain', yTrain[0], '\n')

print('Первый пример xTrain:', xTrain[0][step-5:step+5])
print('Второй пример xTrain:', xTrain[1][:10])

Длина xTrain: 1636
Длина yTrain: 1636
Длина примера из xTrain: 256
Длина примера из yTrain: 256 

Пример xTrain ['Male', 'No', 'No', 'Healthcare', 'Low', 'Cat_4', 'D', 'Female', 'Yes', 'Yes', 'Engineer', 'Average', 'Cat_4', 'A', 'Female', 'Yes', 'Yes', 'Engineer', 'Low', 'Cat_6', 'B', 'Male', 'Yes', 'Yes', 'Lawyer', 'High', 'Cat_6', 'B', 'Female', 'Yes', 'Yes', 'Entertainment', 'High', 'Cat_6', 'A', 'Male', 'Yes', 'No', 'Artist', 'Average', 'Cat_6', 'C', 'Male', 'No', 'Yes', 'Healthcare', 'Low', 'Cat_6', 'C', 'Female', 'No', 'Yes', 'Healthcare', 'Low', 'Cat_6', 'D', 'Female', 'Yes', 'Yes', 'Engineer', 'Low', 'Cat_7', 'D', 'Female', 'Yes', 'Yes', 'Artist', 'Average', 'Cat_6', 'C', 'Female', 'No', 'Yes', 'Engineer', 'Low', 'Cat_6', 'A', 'Male', 'No', 'No', 'Healthcare', 'Low', 'Cat_4', 'D', 'Female', 'No', 'No', 'Executive', 'Low', 'Cat_3', 'D', 'Male', 'Yes', 'No', 'Lawyer', 'Low', 'Cat_6', 'A', 'Female', 'Yes', 'No', 'Doctor', 'Low', 'Cat_3', 'B', 'Female', 'No', 'No', 'Healthcare', 'L

In [ ]:
tok_agreemTest = tokenizer.texts_to_sequences(docTest)

print("Посмотрим на фрагмент тестового текста:")
print("Исходный текст:              ", docTest[:20])
print("Тот же текст, но как последовательность индексов: ", tok_agreemTest[:20], '\n')

xDataTest, yDataTest = getSamples(tok_agreemTest, tags_index, seg_idx)
decoded_text = tegDeleter(docTest)
print('Длина xDataTest:', len(xDataTest))
print('Длина yDataTest:', len(yDataTest))

print('Сдекодированные текст:', decoded_text[50:90])
print('Часть xDataTest:     ', xDataTest[50:90])
print('Часть yDataTest:     ', yDataTest[50:90])

xLen = 256
step = 30 
embeddingSize = 300 
xTest = getSetFromIndices(decoded_text, xLen, step) 
yTest = getSetFromIndices(yDataTest, xLen, step) 

print('Длина xTest:', len(xTest))
print('Длина yTest:', len(yTest))
print('Длина примера xTest:',len(xTest[0]))
print('Длина примера yTrain:',len(yTest[0]), '\n')
print('Пример xTest', xTest[0])
print('Пример yTest', yTest[0], '\n')

print('Первый пример xTest:', xTest[0][step-5:step+5])
print('Второй пример xTest:', xTest[1][:10])

Посмотрим на фрагмент тестового текста:
Исходный текст:               ['D', 'Male', 'Yes', 'Yes', '<s5>', 'Artist', '</s5>', 'Low', 'Cat_7', 'B', 'Female', 'Yes', 'Yes', '<s3>', 'Lawyer', '</s3>', 'High', 'Cat_6', 'C', 'Male']
Тот же текст, но как последовательность индексов:  [[10], [5], [1], [1], [7], [8], [9], [4], [38], [14], [6], [1], [1], [28], [29], [30], [18], [3], [13], [5]] 

Длина xDataTest: 7155
Длина yDataTest: 7155
Сдекодированные текст: ['Male', 'Yes', 'No', 'Doctor', 'Average', 'Cat_6', 'A', 'Female', 'Yes', 'No', 'Marketing', 'Average', 'Cat_4', 'D', 'Male', 'Yes', 'Yes', 'Artist', 'Average', 'Cat_6', 'B', 'Male', 'No', 'No', 'Healthcare', 'Low', 'Cat_6', 'D', 'Male', 'Yes', 'Yes', 'Executive', 'High', 'Cat_5', 'C', 'Male', 'Yes', 'Yes', 'Executive', 'High']
Часть xDataTest:      [[5], [1], [2], [26], [11], [3], [12], [6], [1], [2], [34], [11], [19], [10], [5], [1], [1], [8], [11], [3], [14], [5], [2], [2], [16], [4], [3], [10], [5], [1], [1], [31], [18], [40], [13], [

In [ ]:
def pad_zeros(phrase, xLen=256): 
  while len(phrase) < xLen: 
    phrase.append([0] * embeddingSize)   
  if len(phrase) > xLen: 
    phrase = phrase[:xLen] 
  return phrase 


def getSets(model, senI, tagI):
  xVector = [] 
  tmp = [] 
  for text in senI: 
    tmp=[]
    for word in text: 
      try: 
        tmp.append(model[word])
      except: 
        pass
    xVector.append(pad_zeros(tmp, xLen))
  temp = np.array(xVector)
  return np.array(xVector, dtype = np.float32), np.array(tagI)

In [ ]:
modelGENSIM = word2vec.Word2Vec(xTrain + xTest, size = embeddingSize, window = 10, min_count = 1, workers = 10, iter = 10, max_vocab_size = 1e5)

In [ ]:
print(modelGENSIM['Engineer'])

[ 0.0103555   0.23524001  0.00911836 -0.07201196 -0.10086256 -0.35668844
  0.56353235  0.5750691  -0.01254963 -0.12773757  0.00527211 -0.08594963
 -0.233864   -0.22035944  0.20812413  0.12270918  0.00179409  0.27364257
  0.2119388  -0.34874073 -0.28126925  0.35340768 -0.33640078 -0.03064636
  0.36180416 -0.0215127  -0.2680131   0.04477949  0.11449442 -0.25558123
  0.35076028  0.2272578   0.03322379 -0.0497538  -0.11758063  0.14567186
  0.28779945 -0.01281483 -0.08486577  0.11601359  0.08067676  0.03378317
  0.19696182 -0.27573702  0.1485971   0.09554092 -0.02475788 -0.23880714
  0.03488251  0.07638893  0.06818949 -0.10209714 -0.27245477  0.07514148
  0.07935369  0.21928675  0.01322034  0.05287116  0.30444163  0.26773196
  0.35640585  0.0797385  -0.22146763  0.01619623  0.17541823 -0.3509735
  0.12491547  0.2048233  -0.2147161  -0.369824   -0.40813935  0.03823726
  0.23341991 -0.15356089  0.05908666  0.08441424 -0.0021633  -0.13903598
  0.03553668  0.2575037   0.0414684   0.26396716 -0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [ ]:
GENSIMtrainX, GENSIMtrainY = getSets(modelGENSIM, xTrain, yTrain)
GENSIMtestX, GENSIMtestY = getSets(modelGENSIM, xTest, yTest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


In [ ]:
print('Размерности xTrain:', GENSIMtrainX.shape)
print('Размерности yTrain:', GENSIMtrainY.shape)

Размерности xTrain: (1636, 256, 300)
Размерности yTrain: (1636, 256, 6)


In [ ]:
def dice_coef(y_true, y_pred):
    return (2. * K.sum(y_true * y_pred) + 1.) / (K.sum(y_true) + K.sum(y_pred) + 1.)

In [ ]:
def create_unet(k = 1, num_classes = 6, input_shape= (30, 300)):
    img_input = Input(input_shape) 

    # Блок 1
    x = Conv1D(64 * k , 3, padding='same')(img_input) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(64 * k, 3, padding='same')(x)  
    x = BatchNormalization()(x)     
    block_1_out = Activation('relu')(x) 

    # Блок 2
    x = MaxPooling1D()(block_1_out)
    x = Conv1D(128 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_2_out = Activation('relu')(x)

    # Блок 3
    x = MaxPooling1D()(block_2_out)
    x = Conv1D(256 * k, 3, padding='same')(x)
    x = BatchNormalization()(x)               
    x = Activation('relu')(x)                     

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    block_3_out = Activation('relu')(x)

    # Блок 4
    x = MaxPooling1D()(block_3_out)
    x = Conv1D(512 * k, 3, padding='same')(x)
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(512 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)      
    block_4_out = Activation('relu')(x)
    x = block_4_out 

    # UP 2
    x = Conv1DTranspose(256 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_3_out]) 
    x = Conv1D(256 * k , 3, padding='same')(x) 
    x = BatchNormalization()(x) 
    x = Activation('relu')(x)

    x = Conv1D(256 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # UP 3
    x = Conv1DTranspose(128 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 

    x = concatenate([x, block_2_out])
    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # UP 4
    x = Conv1DTranspose(64 * k, kernel_size=3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = concatenate([x, block_1_out])
    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(64 * k , 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(num_classes, 3, activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) 

    model = Model(img_input, x)
    model.compile(optimizer=Adam(0.0025), 
                  loss='categorical_crossentropy',
                  metrics=[dice_coef])
  
    return model

In [ ]:
model_b_UNET = create_unet(input_shape=(xLen, embeddingSize))
model_b_UNET.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 300)]   0                                            
__________________________________________________________________________________________________
conv1d_17 (Conv1D)              (None, 256, 64)      57664       input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_19 (BatchNo (None, 256, 64)      256         conv1d_17[0][0]                  
__________________________________________________________________________________________________
activation_19 (Activation)      (None, 256, 64)      0           batch_normalization_19[0][0]     
____________________________________________________________________________________________

In [ ]:
history = model_b_UNET.fit(GENSIMtrainX, GENSIMtrainY, validation_data = (GENSIMtestX, GENSIMtestY), epochs=10, batch_size=64) # Feed the images divided among 2 classes to the model

Epoch 1/10
26/26 [==============================] - 63s 2s/step - loss: 0.0859 - dice_coef: 0.2280 - val_loss: 0.1012 - val_dice_coef: 0.3518
Epoch 2/10
26/26 [==============================] - 55s 2s/step - loss: 0.0697 - dice_coef: 0.2070 - val_loss: 0.0777 - val_dice_coef: 0.0303
Epoch 3/10
26/26 [==============================] - 55s 2s/step - loss: 0.0679 - dice_coef: 0.0028 - val_loss: 0.0775 - val_dice_coef: 7.6708e-04
Epoch 4/10
26/26 [==============================] - 56s 2s/step - loss: 0.0668 - dice_coef: 0.0016 - val_loss: 0.0678 - val_dice_coef: 6.5503e-04
Epoch 5/10
26/26 [==============================] - 56s 2s/step - loss: 0.0674 - dice_coef: 8.1907e-04 - val_loss: 0.0652 - val_dice_coef: 6.4824e-04
Epoch 6/10
26/26 [==============================] - 55s 2s/step - loss: 0.0680 - dice_coef: 5.6913e-04 - val_loss: 0.0595 - val_dice_coef: 6.4772e-04
Epoch 7/10
26/26 [==============================] - 55s 2s/step - loss: 0.0683 - dice_coef: 5.5758e-04 - val_loss: 0.0575 - 

In [ ]:
def recognizeSet(XX, YY, model, tags, length, value):
  correct_list = np.array([0] * 6) 
  incorrect_list =  np.array([0] * 6)  
  XX_array = XX
  YY_array = YY
  pred = model.predict(XX_array)
  pred[pred < value] = 0
  pred[pred > value] = 1
  for element in range(YY_array.shape[0]): 
    for word in range(YY_array.shape[1]):  
      for category in range(YY_array.shape[2]):
        if pred[element][word][category] == YY_array[element][word][category]: 
          correct_list[category] += 1 
        else:  
          incorrect_list[category] += 1 
  for i in range(6):
   print("Сеть распознала категорию  '{}' с точностью в {}%".format(tags[i], round(100*correct_list[i]/(correct_list[i] + incorrect_list[i]), 2)))
  total = round(100*np.mean(correct_list/(correct_list + incorrect_list)),2) 
  print("Средняя точность {}%".format(total))

In [ ]:
tags = ['S1', 'S2', 'S3', 'S4', 'S5', 'S6']
recognizeSet(GENSIMtestX, GENSIMtestY, model_b_UNET, tags, xLen, 0.1)

Сеть распознала категорию  'S1' с точностью в 97.31%
Сеть распознала категорию  'S2' с точностью в 98.79%
Сеть распознала категорию  'S3' с точностью в 98.96%
Сеть распознала категорию  'S4' с точностью в 99.05%
Сеть распознала категорию  'S5' с точностью в 95.19%
Сеть распознала категорию  'S6' с точностью в 99.57%
Средняя точность 98.15%
